In [19]:
# Example 02 - Assistant Example
# This is simular to a chat keeping the context
from openai import OpenAI
import json
import time

# Utility function to show each json structure
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Async call to openai to return only when run is complete
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

client = OpenAI()

# Add message to thread
def add_message(thread, prompt, role="user"):
    return client.beta.threads.messages.create(
        thread_id=thread.id,
        role=role,
        content=prompt,
    )


# Creates an Assistant
assistant = client.beta.assistants.create(
    name="History Teacher",
    instructions="You are a personal geography tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
#show_json(assistant)

# Creates a Thread
thread = client.beta.threads.create()
#show_json(thread)

# Add a message to a previoulsy created thread
message = add_message(thread=thread, prompt="Can you rank the 5 most spoken languages in the world (considering the total population that speaks each one)?")
#show_json(message)

# Creates the RUN the actually ties the Agent with the Thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
#show_json(run)

result = wait_on_run(run, thread)
#show_json(result)

# Pulls messages from chat as responses were completed
messages = client.beta.threads.messages.list(thread_id=thread.id)
# Prints the more recent one (indexed as 0)
print (messages.data[0].content[0].text.value)

# Create the second message to append to our thread
message = add_message(thread=thread, prompt="What if we consider only as first language or official language?")


# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)

print ("--------------------------------- SECOND REPONSE")
print (messages.data[0].content[0].text.value)


1. English
2. Mandarin Chinese
3. Hindi
4. Spanish
5. French
---------------------------------
1. Mandarin Chinese
2. Spanish
3. English
4. Hindi
5. Arabic
